In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42): 
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [6]:
learning_rate = 0.001
batch_size = 16
train_size = X_train2.shape[0]

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    #logits = y.op.inputs[0]
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars)
    
    tf.global_variables_initializer().run() 
    
    val_loss = 9999
    
    for i in range(1000):
        #sess.run(training_op, feed_dict={xs: X_train2, ys: y_train2})
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={xs: batch_x, ys: batch_y})
        
        prev_val_loss = val_loss
        val_loss = sess.run(loss, feed_dict={xs: X_valid2, ys: y_valid2})
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, "Accuracy:", accu)
        
        if val_loss > prev_val_loss: # Early stop
            print("Early stop in epoch", i)
            break
    sess.close()

    print("Training Finish!!!")

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 230.341 Accuracy: 0.341082
1 Validation loss: 219.703 Accuracy: 0.476445
2 Validation loss: 211.143 Accuracy: 0.545361
3 Validation loss: 205.506 Accuracy: 0.582802
4 Validation loss: 200.037 Accuracy: 0.612425
5 Validation loss: 192.39 Accuracy: 0.668175
6 Validation loss: 186.899 Accuracy: 0.697799
7 Validation loss: 184.099 Accuracy: 0.71117
8 Validation loss: 181.691 Accuracy: 0.724748
9 Validation loss: 179.81 Accuracy: 0.731948
10 Validation loss: 178.311 Accuracy: 0.739765
11 Validation loss: 177.075 Accuracy: 0.747583
12 Validation loss: 176.033 Accuracy: 0.751903
13 Validation loss: 175.139 Accuracy: 0.757046
14 Validation loss: 174.36 Accuracy: 0.760543
15 Validation loss: 173.674 Accuracy: 0.76404
16 Validation loss: 173.065 Accuracy: 0.767743
17 Validation loss: 172.521 Accuracy: 0.769595
18 Validation loss: 172.032 Accuracy: 0.772886
19 Validation loss: 171.592 Accuracy: 0.775355
20 Validation lo